In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [2]:
import pandas as pd
import requests
from io import BytesIO
from boto3.session import Session
import boto3
import botocore

In [3]:
from data_management.models import *
from loader import load_credential

In [ ]:
ACCESS_KEY = load_credential("AWS_ACCESS_KEY_ID","")
SECRET_ACCESS_KEY = load_credential("AWS_SECRET_ACCESS_KEY","")

In [4]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [12]:
class DataExtractSave():
    def __init__(self):
        self.left = []
        self.top = []
        self.right = []
        self.bottom = []
        self.origin_url = []
        self.width = []
        self.height = []
    
    def get_image_size(self, resp):
        from PIL import Image
        byteImgIO = BytesIO()
        byteImg = Image.open(BytesIO(resp.content))
        byteImg.convert('RGB').save(byteImgIO, "JPEG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        dataBytesIO = BytesIO(byteImg)
        image = Image.open(dataBytesIO)
        image = image.convert('RGB')
        width, height = image.size
        return width, height
    
    def get_data_and_append(self, obj, url):
        self.origin_url.append(url)
        resp = requests.get(url)
        w, h = self.get_image_size(resp)
        self.width.append(w)
        self.height.append(h)
        self.left.append(obj.left*w)
        self.top.append(obj.top*h)
        self.right.append(obj.right*w)
        self.bottom.append(obj.bottom*h)
    
    def save_to_csv(self,  filename):
        dic = {}
        dic['left'] = self.left
        dic['top'] = self.top
        dic['right'] = self.right
        dic['bottom'] = self.bottom
        dic['origin_url'] = self.origin_url
        dic['width'] = self.width
        dic['height'] = self.height
    
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('downloaded')
        
    def save_data_to_csv(self, filename, queryset):       
        print(queryset.count())
        i = 0
        for c in queryset:
            i +=1
            if i%50==0:
                print(i,'/ ',c)
            if c.origin_source.image:
                url = c.origin_source.image.url
                self.get_data_and_append(c, url)
            elif c.origin_source.s3_image_url:
                url = c.origin_source.s3_image_url
                self.get_data_and_append(c, url)
        self.save_to_csv(filename)
        print('DONE!')

In [ ]:
class S3DownloadManager(): # download from queryset
        def __init__(self):
            self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
            self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
            self.s3 =  boto3.resource('s3',
                                    aws_access_key_id=ACCESS_KEY,
                                    aws_secret_access_key=SECRET_ACCESS_KEY)
            self.temp_bucket_list = self.get_s3_file_temp_bucket()
            self.mondedata_bucket_list = self.get_s3_file_mondedata_bucket()
            
        
        def get_s3_file_temp_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('temp-originalimage').objects.all():
                lists.append(s3_file.key)
            return lists
        
        def get_s3_file_mondedata_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('monde-data').objects.all():
                if s3_file.key.split('/')[0] == 'original-bag-images-dev':
                    lists.append(s3_file.key)
            return lists
        
        # s3_image_url 이 존재할 때
        def get_list_from_queryset(self, queryset, n):
            new_file_list_t = []
            new_file_list_m = []
            
            qs_temp_bucket = queryset.filter(s3_image_url__isnull=False)
            temp_bucket_list = []
            qs_mondedata_bucket = queryset.exclude(image="")
            mondedata_bucket_list = []
            
            for q in qs_temp_bucket:
                name = q.s3_image_url.split('/')[-1]
                temp_bucket_list.append(name)
                
            for q in qs_mondedata_bucket:
                name = q.image.name
                mondedata_bucket_list.append(name)
            
            pre_temp_bucket_list = self.temp_bucket_list
            
            #temp_bucket
            for i in range(len(pre_temp_bucket_list)):
                for j in range(len(temp_bucket_list)):
                    if(pre_temp_bucket_list[i] == temp_bucket_list[j]):
                        new_file_list_t.append(pre_temp_bucket_list[i])
        
            pre_mondedata_bucket_list = self.mondedata_bucket_list
            
            #mondedata_bucket
            for i in range(len(pre_mondedata_bucket_list)):
                for j in range(len(mondedata_bucket_list)):
                    if(pre_mondedata_bucket_list[i] == mondedata_bucket_list[j]):
                        new_file_list_m.append(pre_mondedata_bucket_list[i])
                        
            if n == 1: # temp bucket
                return new_file_list_t
            elif n == 2:
                return new_file_list_m
            else:
                print('ERROR')
    
        def download_s3_files_from_queryset(self, local_path, queryset):
            # local_path 는 jupyter를 실행한 폴더 기준
            # 폴더 경로는 \\로 끝나야 함 (for Window)
            file_list_t = self.get_list_from_queryset(queryset, 1)
            file_list_m = self.get_list_from_queryset(queryset, 2)
            i = 0
            for file in file_list_t:
                i += 1
                KEY = file
                dest_pathname = os.path.join(local_path, KEY)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('temp-originalimage').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print('======')
        
            i = 0
            for file in file_list_m:
                i += 1
                KEY = file
                path_key = KEY.split('/')[1]
                dest_pathname = os.path.join(local_path, path_key)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('monde-data').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print('DONE')

In [ ]:
# How to Use

In [11]:
# instance생성 및 queryset 선언
extractmanager1= DataExtractSave()
imagedownloadmanager1 = S3DownloadManager()
queryset1 = CroppedImage.objects.filter(origin_source__image_url__isnull=False)

In [ ]:
# queryset info download csv form
extractmanager1.save_data_to_csv("crop_download_test.csv", queryset1)

In [ ]:
# image download to 'data' folder => 다운받기 전 'data'라는 폴더 생성해야 합니다.
imagedownloadmanager1.download_s3_files_from_queryset('./data/', queryset1)

In [ ]:
########### image review (몸체만 정확히 박싱된 데이터) extract

In [15]:
extractmanager2= DataExtractSave()
imagedownloadmanager2 = S3DownloadManager()
queryset2 = CroppedImage.objects.filter(origin_source__image_review=True)

In [ ]:
extractmanager2.save_data_to_csv("20191106_tight_boxed_data.csv", queryset2)
imagedownloadmanager2.download_s3_files_from_queryset('./data/', queryset2)

In [ ]:
# TODO : 현재는 박싱 서버에서 쌓은 데이터 기반으로 사용 가능하지만, 추후 mondebro에서 태깅한 데이터도 사용 가능하게 수정해야 합니다.